<a href="https://colab.research.google.com/github/jbtuku/Data-Engineering/blob/main/Data%20Synthsesis/datasynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from google.colab import files

# STEP 3: Upload Excel file
print("  upload  'Departments Roles and Salaries.xlsx'")
uploaded = files.upload()
excel_file = list(uploaded.keys())[0]

# STEP 4: Set seed
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# STEP 5: Read both sheets
departments_df = pd.read_excel(excel_file, sheet_name=0)
roles_df = pd.read_excel(excel_file, sheet_name=1)

roles_df.columns = [col.strip() for col in roles_df.columns]
departments_df.columns = [col.strip() for col in departments_df.columns]

departments_df['% of employees'] = departments_df['% of employees'].astype(str).str.replace('%', '').astype(float)

departments_df['weight'] = departments_df['% of employees'] / departments_df['% of employees'].sum()


departments = departments_df['Department'].tolist()
weights = departments_df['weight'].tolist()


n = 10000
faker_us = Faker('en_US')

country_distribution = {
    'USA': 60.0,
    'India': 17.0,
    'China': 11.8,
    'Mexico': 4.5,
    'Canada': 2.7,
    'Philippines': 1.8,
    'Taiwan': 1.2,
    'South Korea': 1.0
}
total = sum(country_distribution.values())
country_list = list(country_distribution.keys())
country_weights = [v / total for v in country_distribution.values()]

locale_map = {
    'USA': 'en_US', 'India': 'en_IN', 'China': 'zh_CN', 'Mexico': 'es_MX',
    'Canada': 'en_CA', 'Philippines': 'en_PH', 'Taiwan': 'zh_TW', 'South Korea': 'ko_KR'
}
locale_fakers = {c: Faker(locale_map[c]) for c in country_list}

employee_ids = np.arange(100000000, 100000000 + n)
country_choices = random.choices(country_list, weights=country_weights, k=n)
names, phones, emails = [], [], []

for c in country_choices:
    f = locale_fakers[c]
    names.append(f.name())
    emails.append(f.email())
    phones.append(faker_us.phone_number())


chosen_departments = random.choices(departments, weights=weights, k=n)
chosen_roles = []
salaries = []

for dept in chosen_departments:
    role_subset = roles_df[roles_df['Department'] == dept]
    if not role_subset.empty:
        chosen = role_subset.sample(1).iloc[0]
        chosen_roles.append(chosen['Role'])
        salary = random.randint(int(chosen['Lower']), int(chosen['Upper']))
        salaries.append(salary)
    else:
        chosen_roles.append("Unassigned")
        salaries.append(0)


emp_df = pd.DataFrame({
    'employeeID': employee_ids,
    'CountryOfBirth': country_choices,
    'name': names,
    'phone': phones,
    'email': emails,
    'department': chosen_departments,
    'role': chosen_roles,
    'salary': salaries
})


print(" emp_df.head(10):")
display(emp_df.head(10))

print(" emp_df.describe(include='all'):")
display(emp_df.describe(include='all'))

total_payroll = emp_df['salary'].sum()
print(f"\n Total Yearly Payroll: ${total_payroll:,.2f}")

  upload  'Departments Roles and Salaries.xlsx'


Saving Departments Roles and Salaries.xlsx to Departments Roles and Salaries.xlsx
 emp_df.head(10):


,employeeID,CountryOfBirth,name,phone,email,department,role,salary
0,100000000,India,Aryan Maharaj,581-296-0013,udantdewan@example.net,Legal,Legal Operations,157791
1,100000001,USA,Michelle Miles,3268542351,lrobinson@example.com,Human Resources,Senior Recruiter,120367
2,100000002,USA,Ryan Munoz,584-695-9310,blairamanda@example.com,Customer Support,Customer Support Representative,56377
3,100000003,USA,Dr. Sharon James,(653)541-9283x276,xreid@example.org,Customer Support,Customer Support Representative,63287
4,100000004,India,Daksh Karnik,639-453-7672,muttinandini@example.com,Engineering,Full-Stack Engineer,132216
5,100000005,India,Adya Sangha,671.201.2269x16697,opathak@example.org,Administrative,IT Services Administrative Assistant,61280
6,100000006,Mexico,Omar Jaramillo Molina,704.482.8148x932,lozadamanuel@example.org,Legal,Legal Operations,116691
7,100000007,USA,Paula Moreno,(443)903-9117x182,georgetracy@example.org,Sales,Sales Development Rep,70406
8,100000008,USA,Carmen Rose,001-734-265-7871x331,ibrandt@example.net,Marketing,Marketing Manager,95097
9,100000009,USA,Crystal Robinson,4519834738,zimmermanbrian@example.org,Engineering,Frontend Engineer,105044


 emp_df.describe(include='all'):


,employeeID,CountryOfBirth,name,phone,email,department,role,salary
count,1.000000e+04,10000,10000,10000,10000,10000,10000,10000.000000
unique,NaN,8,9592,10000,9871,11,75,NaN
top,NaN,USA,James Jones,(818)826-0176x2222,jamessmith@example.com,Engineering,Backend Engineer,NaN
freq,NaN,5957,4,1,3,2980,531,NaN
mean,1.000050e+08,NaN,NaN,NaN,NaN,NaN,NaN,157153.895500
std,2.886896e+03,NaN,NaN,NaN,NaN,NaN,NaN,84703.503262
min,1.000000e+08,NaN,NaN,NaN,NaN,NaN,NaN,35072.000000
25%,1.000025e+08,NaN,NaN,NaN,NaN,NaN,NaN,96930.250000
50%,1.000050e+08,NaN,NaN,NaN,NaN,NaN,NaN,134655.500000
75%,1.000075e+08,NaN,NaN,NaN,NaN,NaN,NaN,209598.750000



 Total Yearly Payroll: $1,571,538,955.00
